In [5]:
import pandas as pd

In [6]:
# read in the metro long data
zori = pd.read_csv('data/zori_metro_long_clean.csv')

# change the date to datetime
zori['date'] = pd.to_datetime(zori['date'])

# create a yoy % change column for zori in each metro
zori['zori_yoy'] = zori.groupby('name')['zori'].pct_change(12)

zori

,name,date,zori,lat,lng,pop_2023,zori_yoy
0,"New York, NY",2015-02-01,2255.133793,40.6943,-73.9249,19498249.0,NaN
1,"Los Angeles, CA",2015-02-01,1814.136486,34.1141,-118.4068,12799100.0,NaN
2,"Chicago, IL",2015-02-01,1356.915213,41.8375,-87.6866,9262825.0,NaN
3,"Dallas, TX",2015-02-01,1090.919667,32.7935,-96.7667,8100037.0,NaN
4,"Houston, TX",2015-02-01,1229.410303,29.7860,-95.3885,7510253.0,NaN
...,...,...,...,...,...,...,...
5945,"Salt Lake City, UT",2024-12-01,1650.452457,40.7776,-111.9311,1267864.0,0.012898
5946,"Hartford, CT",2024-12-01,1874.958152,41.7661,-72.6834,1151543.0,0.075844
5947,"Buffalo, NY",2024-12-01,1344.539267,42.9018,-78.8487,1155604.0,0.058990
5948,"Birmingham, AL",2024-12-01,1365.603146,33.5279,-86.7971,1184290.0,0.046951


In [7]:
demand_link = "https://files.zillowstatic.com/research/public_csvs/zordi/Metro_zordi_uc_sfrcondomfr_month.csv?t=1734717130"

demand = pd.read_csv(demand_link)

demand

,RegionID,SizeRank,RegionName,RegionType,StateName,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30
0,102001,0,United States,country,NaN,100.0,102.0,95.0,85.0,74.0,...,73.0,76.0,78.0,78.0,75.0,66.0,60.0,53.0,48.0,43.0
1,394913,1,"New York, NY",msa,NY,53.0,51.0,53.0,58.0,58.0,...,111.0,112.0,113.0,111.0,108.0,106.0,102.0,97.0,90.0,85.0
2,753899,2,"Los Angeles, CA",msa,CA,134.0,132.0,116.0,96.0,81.0,...,91.0,87.0,88.0,90.0,91.0,84.0,77.0,69.0,62.0,54.0
3,394463,3,"Chicago, IL",msa,IL,97.0,91.0,72.0,50.0,39.0,...,141.0,150.0,151.0,149.0,143.0,129.0,116.0,101.0,90.0,81.0
4,394514,4,"Dallas, TX",msa,TX,108.0,104.0,91.0,80.0,72.0,...,52.0,53.0,53.0,51.0,46.0,37.0,30.0,27.0,24.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,395104,926,"Snyder, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,...,14.0,12.0,10.0,10.0,10.0,11.0,15.0,15.0,8.0,1.0
847,394342,929,"Atchison, KS",msa,KS,NaN,NaN,NaN,NaN,NaN,...,NaN,62.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848,753880,930,"Fairfield, IA",msa,IA,NaN,NaN,NaN,NaN,NaN,...,20.0,19.0,18.0,13.0,NaN,NaN,NaN,NaN,NaN,5.0
849,395186,932,"Vermillion, SD",msa,SD,NaN,NaN,NaN,NaN,NaN,...,50.0,44.0,42.0,44.0,48.0,32.0,26.0,34.0,54.0,NaN


In [8]:
# read in demand data
demand = pd.read_csv(demand_link)
                     
# only msa
demand = demand[demand['RegionType'] == 'msa']

# drop cols I don't need
demand.drop(
    columns=['RegionID', 'SizeRank', 'RegionType'],
    inplace=True
)

# rename name
demand = demand.rename(columns={
    'RegionName': 'name',
    'StateName': 'state'
    }
    )

#  melt the data
demand = demand.melt(
    id_vars=['name', 'state'],
    var_name='date', 
    value_name='demand'
    )

# change 'date' from the last day of the month to the first day of the month
demand['date'] = pd.to_datetime(demand['date'])
demand['date'] = demand['date'] + pd.offsets.MonthBegin(-1)

demand


,name,state,date,demand
0,"New York, NY",NY,2020-06-01,53.0
1,"Los Angeles, CA",CA,2020-06-01,134.0
2,"Chicago, IL",IL,2020-06-01,97.0
3,"Dallas, TX",TX,2020-06-01,108.0
4,"Houston, TX",TX,2020-06-01,46.0
...,...,...,...,...
45895,"Snyder, TX",TX,2024-11-01,1.0
45896,"Atchison, KS",KS,2024-11-01,NaN
45897,"Fairfield, IA",IA,2024-11-01,5.0
45898,"Vermillion, SD",SD,2024-11-01,NaN


In [9]:
# merge the data
demand_rent = pd.merge(
    zori,
    demand,
    on=['name', 'date'],
    how='inner'
    )

demand_rent 

,name,date,zori,lat,lng,pop_2023,zori_yoy,state,demand
0,"New York, NY",2020-06-01,2555.063031,40.6943,-73.9249,19498249.0,0.006697,NY,53.0
1,"Los Angeles, CA",2020-06-01,2322.129994,34.1141,-118.4068,12799100.0,0.010358,CA,134.0
2,"Chicago, IL",2020-06-01,1571.460683,41.8375,-87.6866,9262825.0,0.013367,IL,97.0
3,"Dallas, TX",2020-06-01,1360.951027,32.7935,-96.7667,8100037.0,0.015752,TX,108.0
4,"Houston, TX",2020-06-01,1372.666017,29.7860,-95.3885,7510253.0,0.009876,TX,46.0
...,...,...,...,...,...,...,...,...,...
2695,"Salt Lake City, UT",2024-11-01,1665.650706,40.7776,-111.9311,1267864.0,0.016908,UT,28.0
2696,"Hartford, CT",2024-11-01,1876.735406,41.7661,-72.6834,1151543.0,0.072057,CT,119.0
2697,"Buffalo, NY",2024-11-01,1340.183994,42.9018,-78.8487,1155604.0,0.053459,NY,95.0
2698,"Birmingham, AL",2024-11-01,1364.750631,33.5279,-86.7971,1184290.0,0.046250,AL,33.0


In [18]:
us_regions = {
    "West" : ["WA", "OR", "CA", "NV", "ID", "MT", "WY", "UT", "CO", "HI", "AK"],
    "Southwest" : ["AZ", "NM", "TX", "OK"],
    "Midwest" : ["ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "IN", "MI", "OH"],
    "Southeast" : ["AR", "LA", "MS", "AL", "TN", "KY", "FL", "GA", "SC", "NC", "VA", "WV"],
    "Northest" : ["ME", "NH", "VT", "MA", "RI", "CT", "NY", "NJ", "PA", "DE", "MD", "DC"]
}

In [23]:
# create a regions df with cols 'state' and 'region' using the us_regions dict
regions = pd.DataFrame(
    [(state, region) for region, states in us_regions.items() for state in states],
    columns=['state', 'region']
    )

regions.to_csv('data/regions.csv', index=False)

In [19]:
# Assign regions to rows based on the dictionary above
demand_rent['region'] = demand_rent['state'].map({state: region for region, states in us_regions.items() for state in states})

demand_rent

,name,date,zori,lat,lng,pop_2023,zori_yoy,state,demand,region
0,"New York, NY",2020-06-01,2555.063031,40.6943,-73.9249,19498249.0,0.006697,NY,53.0,Northest
1,"Los Angeles, CA",2020-06-01,2322.129994,34.1141,-118.4068,12799100.0,0.010358,CA,134.0,West
2,"Chicago, IL",2020-06-01,1571.460683,41.8375,-87.6866,9262825.0,0.013367,IL,97.0,Midwest
3,"Dallas, TX",2020-06-01,1360.951027,32.7935,-96.7667,8100037.0,0.015752,TX,108.0,Southwest
4,"Houston, TX",2020-06-01,1372.666017,29.7860,-95.3885,7510253.0,0.009876,TX,46.0,Southwest
...,...,...,...,...,...,...,...,...,...,...
2695,"Salt Lake City, UT",2024-11-01,1665.650706,40.7776,-111.9311,1267864.0,0.016908,UT,28.0,West
2696,"Hartford, CT",2024-11-01,1876.735406,41.7661,-72.6834,1151543.0,0.072057,CT,119.0,Northest
2697,"Buffalo, NY",2024-11-01,1340.183994,42.9018,-78.8487,1155604.0,0.053459,NY,95.0,Northest
2698,"Birmingham, AL",2024-11-01,1364.750631,33.5279,-86.7971,1184290.0,0.046250,AL,33.0,Southeast


In [13]:
demand_rent.state.unique()

array(['NY', 'CA', 'IL', 'TX', 'VA', 'PA', 'FL', 'GA', 'MA', 'AZ', 'MI',
       'WA', 'MN', 'CO', 'MD', 'MO', 'NC', 'OR', 'OH', 'NV', 'IN', 'TN',
       'RI', 'WI', 'OK', 'KY', 'UT', 'CT', 'AL'], dtype=object)

In [24]:
# save the data
demand_rent.to_csv('data/demand_rent_metro.csv', index=False)